<a href="https://colab.research.google.com/github/Manuel-Gomez-05/SenalesySistemas2/blob/main/Parcial_2_SyS_2025_02_Manuel_Alejandro_Gomez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Solucion_Parcial_2_SyS_2025_II_Manuel_Gomez.ipynb"""

import os
import subprocess
import time
import re

# --- 1. Instalación de Dependencias ---
print("--- Iniciando configuración e instalación de librerías ---")
print("Esto puede tardar unos segundos...")
subprocess.run([
    "pip", "install", "streamlit", "numpy", "scipy", "matplotlib", "pandas",
    "soundfile", "pydub", "yt-dlp", "-q"
], check=True)

# Instalar ffmpeg (necesario para procesar audio con pydub)
subprocess.run(["apt-get", "install", "ffmpeg", "-y", "-qq"], check=True)

# --- 2. Crear estructura de carpetas ---
print("\n--- Configurando directorios del proyecto ---")
if not os.path.exists("pages"):
    os.makedirs("pages")

# ==========================================
# ARCHIVO 1: PRESENTACIÓN (HOME)
# ==========================================
with open("presentacion.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st

st.set_page_config(
    page_title="Parcial 2 - SyS 2025-II",
    page_icon="📡",
    layout="wide"
)

st.title("Parcial 2: Señales y Sistemas 2025-II")
st.markdown("**Profesor:** Andrés Marino Álvarez Meza, Ph.D.")
st.markdown("### **Estudiante:** Manuel Alejandro Gómez")
st.markdown("---")

st.info("""
👈 **Utilice el menú lateral** para navegar entre los puntos del examen.
""")

col1, col2 = st.columns(2)

with col1:
    st.header("Punto 1: Modulación AM")
    st.markdown("""
    **Simulación de Modulación y Demodulación DSB-SC.**
    * Señal de entrada: Audio real (Youtube/Archivo) o Tono de prueba.
    * Modulación: Multiplicación por portadora $cos(2\pi f_c t)$.
    * Demodulación: Coherente (mezclador) + **Filtro FFT**.
    * **Requisito cumplido:** Filtrado mediante manipulación directa del espectro (FFT) para eliminar réplicas en alta frecuencia.
    """)

with col2:
    st.header("Punto 2: Sistemas Dinámicos")
    st.markdown("""
    **Equivalencia Mecánico-Eléctrico y Control.**
    * Sistema Masa-Resorte-Amortiguador vs. Circuito RLC.
    * Análisis de respuesta transitoria (Sub, Crítico, Sobre).
    * **Análisis de Lazo Cerrado:** Implementación de control con Ganancia Proporcional ($K_p$) para evidenciar cambios en la dinámica del sistema.
    """)
''')

# ==========================================
# ARCHIVO 2: PUNTO 1 (MODULACIÓN CON FFT)
# ==========================================
with open("pages/punto1_modulacion.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft, fftfreq, fftshift
import yt_dlp
from pydub import AudioSegment

st.set_page_config(page_title="Punto 1: Modulación", layout="wide")

# --- Funciones de utilidad ---
def download_youtube_audio(url, fs=44100):
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': 'temp_audio.%(ext)s',
            'postprocessors': [{'key': 'FFmpegExtractAudio','preferredcodec': 'wav',}],
            'quiet': True,
        }
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        audio = AudioSegment.from_file("temp_audio.wav")
        audio = audio.set_channels(1).set_frame_rate(fs)
        # Cortar 5 segundos exactos si es posible
        if len(audio) > 5000:
            audio = audio[30000:35000] # Tomar del seg 30 al 35
        data = np.array(audio.get_array_of_samples()).astype(np.float32)
        # Normalizar
        if np.max(np.abs(data)) > 0:
            data = data / np.max(np.abs(data))
        return fs, data
    except Exception as e:
        st.error(f"Error descargando: {e}")
        return None, None

def plot_signal_spectrum(t, y, fs, title, color='blue'):
    """Grafica tiempo y frecuencia usando FFT"""
    N = len(y)
    Y = fft(y)
    freqs = fftfreq(N, 1/fs)

    # Solo mitad positiva para visualizar mejor
    Y_shift = fftshift(Y)
    freqs_shift = fftshift(freqs)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 3.5))

    # Tiempo
    ax1.plot(t, y, color=color, lw=1)
    ax1.set_title(f"{title} - Tiempo")
    ax1.set_xlabel("Tiempo (s)")
    ax1.grid(True, alpha=0.3)

    # Frecuencia
    ax2.plot(freqs_shift, np.abs(Y_shift)/N, color='red', lw=1)
    ax2.set_title(f"{title} - Espectro (Amplitud)")
    ax2.set_xlabel("Frecuencia (Hz)")
    ax2.set_xlim([-fs/2, fs/2])
    ax2.grid(True, alpha=0.3)

    st.pyplot(fig)

# --- Interfaz ---
st.title("1. Demodulación AM (DSB-SC) con Filtro FFT")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")
st.markdown("Implementación de la Figura 1 del parcial. Filtrado realizado en el dominio de la frecuencia.")

# Sidebar Controls
with st.sidebar:
    st.header("Parámetros")
    source_opt = st.radio("Fuente de Audio", ["Generar Tono (Prueba)", "YouTube", "Archivo Local"])
    fc = st.slider("Frecuencia Portadora (fc)", 5000, 15000, 10000, step=1000)
    cutoff_lpf = st.slider("Corte Filtro LPF (Hz)", 1000, 4500, 3000, step=100)

# 1. Adquisición
fs = 44100
audio_data = None
t = None

if source_opt == "YouTube":
    url = st.text_input("URL Youtube", "https://www.youtube.com/watch?v=dQw4w9WgXcQ")
    if st.button("Cargar Audio"):
        with st.spinner("Descargando..."):
            fs, audio_data = download_youtube_audio(url, fs)
elif source_opt == "Archivo Local":
    upl = st.file_uploader("Subir WAV", type=['wav'])
    if upl:
        audio = AudioSegment.from_file(upl)
        audio = audio.set_channels(1).set_frame_rate(fs)
        if len(audio) > 5000: audio = audio[:5000]
        audio_data = np.array(audio.get_array_of_samples()).astype(np.float32)
        audio_data /= np.max(np.abs(audio_data))
else:
    # Tono puro para pruebas
    t_full = np.linspace(0, 5, 5*fs)
    # Una mezcla de tonos para ver mejor el espectro
    audio_data = 0.5 * np.cos(2*np.pi*440*t_full) + 0.3 * np.cos(2*np.pi*800*t_full)
    audio_data = audio_data * np.exp(-t_full) # Amortiguado

if audio_data is not None:
    t = np.linspace(0, len(audio_data)/fs, len(audio_data))

    st.subheader("1. Mensaje Original $m(t)$")
    st.audio(audio_data, sample_rate=fs)
    plot_signal_spectrum(t, audio_data, fs, "Mensaje m(t)")

    # 2. Modulación
    carrier = np.cos(2 * np.pi * fc * t)
    modulated_signal = audio_data * carrier

    st.subheader("2. Señal Modulada (DSB-SC) $r(t)$")
    st.markdown(r"Señal recibida: $r(t) = A_c m(t) \cos(2\pi f_c t)$")
    st.audio(modulated_signal, sample_rate=fs)
    plot_signal_spectrum(t, modulated_signal, fs, "Modulada r(t)", color='green')

    # 3. Demodulación - Mezcla
    mixer_output = modulated_signal * carrier

    st.subheader("3. Salida del Mezclador (Mixer)")
    st.markdown(r"Antes del filtro: Componente banda base + Componente en $2f_c$")
    plot_signal_spectrum(t, mixer_output, fs, "Mixer Output", color='orange')

    # 4. Demodulación - Filtro Ideal FFT
    st.subheader("4. Filtrado Pasa Bajas (Método FFT)")
    st.markdown("Se aplica una máscara ideal en el dominio de la frecuencia (FFT).")

    # Paso A: FFT
    Y_mixer = fft(mixer_output)
    freqs_mixer = fftfreq(len(mixer_output), 1/fs)

    # Paso B: Crear Máscara Ideal (Rectangular)
    mask = np.abs(freqs_mixer) < cutoff_lpf

    # Paso C: Filtrar
    Y_filtered = Y_mixer * mask

    # Visualizar lo que hizo el filtro en frecuencia
    fig_filt, ax_filt = plt.subplots(figsize=(8,3))
    ax_filt.plot(fftshift(freqs_mixer), fftshift(np.abs(Y_mixer))/len(mixer_output), label="Entrada Filtro", color='lightgray')
    ax_filt.plot(fftshift(freqs_mixer), fftshift(np.abs(Y_filtered))/len(mixer_output), label="Salida Filtro", color='red', alpha=0.7)
    ax_filt.set_xlim([-fc*2.5, fc*2.5])
    ax_filt.legend()
    ax_filt.set_title("Efecto del Filtro FFT en el Espectro")
    st.pyplot(fig_filt)

    # Paso D: IFFT (Recuperar tiempo)
    recovered_signal = ifft(Y_filtered).real * 2 # x2 para recuperar amplitud

    st.subheader("5. Señal Recuperada Final")
    st.audio(recovered_signal, sample_rate=fs)
    plot_signal_spectrum(t, recovered_signal, fs, "Recuperada Final", color='purple')

    st.success("Análisis completo: El filtro FFT eliminó exitosamente la portadora de alta frecuencia.")
''')

# ==========================================
# ARCHIVO 3: PUNTO 2 (SISTEMAS DINÁMICOS)
# ==========================================
with open("pages/punto2_sistemas.py", "w", encoding="utf-8") as f:
    f.write('''
import streamlit as st
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt

st.set_page_config(page_title="Punto 2: Sistemas", layout="wide")

st.title("2. Sistemas Dinámicos y Equivalencias")
st.markdown("**Estudiante:** Manuel Alejandro Gómez")

# --- Teoría ---
with st.expander("Ver Análisis Teórico (Ecuaciones)"):
    st.latex(r"""
    \text{Sistema Mecánico:} \quad H_{mec}(s) = \frac{1}{ms^2 + cs + k}
    """)
    st.latex(r"""
    \text{Sistema Eléctrico (RLC serie L, paralelo RC):} \quad H_{elec}(s) = \frac{1/LC}{s^2 + \frac{1}{RC}s + \frac{1}{LC}}
    """)
    st.markdown("**Equivalencia:** $LC = m/k$ y $RC = m/c$.")

# --- Controles ---
col_in, col_res = st.columns([1, 2])

with col_in:
    st.subheader("Configuración")
    m = st.number_input("Masa (m) [kg]", value=1.0, step=0.1)
    k = st.number_input("Constante Resorte (k) [N/m]", value=10.0, step=1.0) # Valor bajo para ver efecto Kp

    c_crit = 2 * np.sqrt(k*m)
    st.caption(f"Amortiguamiento Crítico base (c_crit): {c_crit:.2f}")

    tipo_amort = st.selectbox("Caso de Amortiguamiento",
                              ["Subamortiguado (zeta < 1)",
                               "Crítico (zeta = 1)",
                               "Sobreamortiguado (zeta > 1)"])

    if "Sub" in tipo_amort: zeta_target = 0.3
    elif "Crítico" in tipo_amort: zeta_target = 1.0
    else: zeta_target = 2.0

    c = st.slider("Coeficiente Amortiguador (c)", 0.0, c_crit*4, c_crit*zeta_target)

    st.markdown("---")
    st.subheader("Estrategia de Control")
    lazo = st.radio("Tipo de Lazo", ["Lazo Abierto", "Lazo Cerrado con Ganancia (Control P)"])

# --- CÁLCULOS Y LÓGICA DE CONTROL ---

# 1. Definir Planta (Lazo Abierto)
# G(s) = 1 / (ms^2 + cs + k)
num_plant = [1]
den_plant = [m, c, k]

Kp = 1.0 # Valor por defecto

if lazo == "Lazo Cerrado con Ganancia (Control P)":
    # Agregamos control proporcional para hacer evidente el cambio
    st.sidebar.markdown("### Controlador")
    Kp = st.sidebar.slider("Ganancia Proporcional (Kp)", 1.0, 100.0, 50.0)

    # Teoría visual
    st.info(f"**Control P Activo:** Se aplica una ganancia $K_p = {Kp}$.\\n"
            f"Esto aumenta la rigidez virtual del sistema: $k_{{equiv}} = k + K_p$.")

    # H_cl = (Kp * G) / (1 + Kp * G) -> Simplificando para G de orden 2:
    # H_cl = Kp / (m*s^2 + c*s + (k + Kp))
    num = [Kp]
    den = [m, c, k + Kp]

else:
    # Lazo Abierto
    st.sidebar.markdown("### Sistema Natural")
    st.sidebar.info("Sistema sin realimentación.")
    num = num_plant
    den = den_plant

# Crear sistema LTI
sys = signal.TransferFunction(num, den)

# Calcular nuevos parámetros dinámicos reales del sistema actual
wn_sys = np.sqrt(den[2] / den[0]) # raiz(k_equiv / m)
zeta_sys = den[1] / (2 * np.sqrt(den[2] * den[0])) # c / 2*raiz(k_equiv*m)

# Equivalentes Eléctricos (Basados en la planta física m,c,k original)
C_elec = 10e-6
L_elec = m / (k * C_elec)
R_elec = m / (c * C_elec) if c > 0 else np.inf

# --- Resultados ---
with col_res:
    st.subheader("Resultados del Análisis")

    # Métricas
    c1, c2, c3 = st.columns(3)
    c1.metric("Zeta Real (ζ)", f"{zeta_sys:.3f}")
    c2.metric("Omega Natural (ωn)", f"{wn_sys:.2f} rad/s")
    c3.metric("Ganancia DC", f"{num[0]/den[2]:.3f}")

    st.markdown(f"**Equivalencia Eléctrica (Planta):** R = {R_elec:.1f} $\Omega$, L = {L_elec:.2f} H, C = 10 $\mu F$")

    # Pestañas de Gráficas
    tab1, tab2, tab3 = st.tabs(["Respuesta al Escalón", "Diagrama de Bode", "Polos y Ceros"])

    t_sim = np.linspace(0, 15, 2000)

    with tab1:
        t_out, y_out = signal.step(sys, T=t_sim)

        fig_time, ax_time = plt.subplots(figsize=(8, 4))
        ax_time.plot(t_out, y_out, lw=2, label=f"Salida y(t) [Kp={Kp}]")
        ax_time.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label="Referencia (Input=1)")

        # Marcar tiempo pico si existe sobrepaso
        if zeta_sys < 1:
            idx_max = np.argmax(y_out)
            ax_time.plot(t_out[idx_max], y_out[idx_max], 'rx')
            ax_time.text(t_out[idx_max], y_out[idx_max]+0.05, f"Mp", color='red')

        ax_time.set_title("Respuesta Transitoria")
        ax_time.set_xlabel("Tiempo (s)")
        ax_time.set_ylabel("Amplitud")
        ax_time.legend()
        ax_time.grid(True, alpha=0.3)
        st.pyplot(fig_time)

        if lazo == "Lazo Cerrado con Ganancia (Control P)":
            st.success("Observe cómo al aumentar Kp, el sistema responde más rápido (ωn sube) y el error de estado estacionario disminuye (la gráfica sube más), aunque disminuye el amortiguamiento.")

    with tab2:
        w, mag, phase = signal.bode(sys)
        fig_bode, (ax_mag, ax_phase) = plt.subplots(2, 1, sharex=True, figsize=(8, 5))
        ax_mag.semilogx(w, mag)
        ax_mag.set_ylabel("Magnitud (dB)")
        ax_mag.grid(True, which='both', alpha=0.3)
        ax_phase.semilogx(w, phase)
        ax_phase.set_ylabel("Fase (deg)")
        ax_phase.set_xlabel("Frecuencia (rad/s)")
        ax_phase.grid(True, which='both', alpha=0.3)
        st.pyplot(fig_bode)

    with tab3:
        # Mapa de Polos y Ceros
        fig_pz, ax_pz = plt.subplots(figsize=(5, 5))

        # Polos
        poles = sys.poles
        ax_pz.scatter(np.real(poles), np.imag(poles), s=100, marker='x', color='red', label='Polos')

        # Ceros (si hubiera)
        if len(sys.zeros) > 0:
            ax_pz.scatter(np.real(sys.zeros), np.imag(sys.zeros), s=100, marker='o', facecolors='none', edgecolors='black', label='Ceros')

        ax_pz.axhline(0, color='black', lw=1)
        ax_pz.axvline(0, color='black', lw=1)
        ax_pz.set_xlabel('Real')
        ax_pz.set_ylabel('Imaginario')
        ax_pz.set_title(f"Mapa de Polos (Lugar de Raíces para Kp={Kp})")
        ax_pz.grid(True, which='both')
        ax_pz.legend()
        st.pyplot(fig_pz)
''')

# --- 3. Ejecución del Tunnel ---
print("\n--- Configurando Tunnel Cloudflare ---")
# Descargar cloudflared si no existe
if not os.path.exists("cloudflared"):
    subprocess.run(["wget", "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64", "-O", "cloudflared"], check=True)
    subprocess.run(["chmod", "+x", "cloudflared"], check=True)

print("--- Ejecutando Streamlit ---")
# Ejecutar Streamlit en background
proc_streamlit = subprocess.Popen(
    ["streamlit", "run", "presentacion.py", "--server.port", "8501", "--server.address", "0.0.0.0"],
    stdout=subprocess.PIPE, stderr=subprocess.PIPE
)

# Ejecutar Cloudflared
# Nota: Usamos trycloudflare que no requiere token, pero rota URLs.
with open("tunnel.log", "w") as log_file:
    proc_tunnel = subprocess.Popen(
        ["./cloudflared", "tunnel", "--url", "http://localhost:8501"],
        stdout=log_file, stderr=subprocess.STDOUT
    )

print("Esperando generación de URL pública (aprox 10 seg)...")
time.sleep(10)

url_publica = None
try:
    with open("tunnel.log", "r") as f:
        # Buscar la URL en el log con regex
        content = f.read()
        match = re.search(r'https://[a-zA-Z0-9-]+\.trycloudflare\.com', content)
        if match:
            url_publica = match.group(0)
except Exception as e:
    print(f"Error leyendo log: {e}")

if url_publica:
    print(f"\n TU APLICACIÓN ESTÁ LISTA. Haz clic aquí:\n{url_publica}\n")
else:
    print("\n⚠️ No se encontró la URL aún. Revisa 'tunnel.log' o espera un momento más.")
    # Fallback para debug
    with open("tunnel.log", "r") as f:
        print("Log dump:", f.read()[-500:])

print("La aplicación seguirá corriendo. Detén la celda para finalizar.")

# Loop infinito para mantener vivo el proceso
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    proc_streamlit.terminate()
    proc_tunnel.terminate()
    print("Procesos detenidos.")

<>:54: SyntaxWarning: invalid escape sequence '\p'
<>:185: SyntaxWarning: invalid escape sequence '\c'
<>:247: SyntaxWarning: invalid escape sequence '\q'
<>:54: SyntaxWarning: invalid escape sequence '\p'
<>:185: SyntaxWarning: invalid escape sequence '\c'
<>:247: SyntaxWarning: invalid escape sequence '\q'
/tmp/ipython-input-3523980814.py:54: SyntaxWarning: invalid escape sequence '\p'
  * Modulación: Multiplicación por portadora $cos(2\pi f_c t)$.
/tmp/ipython-input-3523980814.py:185: SyntaxWarning: invalid escape sequence '\c'
  st.markdown(r"Señal recibida: $r(t) = A_c m(t) \cos(2\pi f_c t)$")
/tmp/ipython-input-3523980814.py:247: SyntaxWarning: invalid escape sequence '\q'
  \text{Sistema Mecánico:} \quad H_{mec}(s) = \frac{1}{ms^2 + cs + k}


--- Iniciando configuración e instalación de librerías ---
Esto puede tardar unos segundos...

--- Configurando directorios del proyecto ---

--- Configurando Tunnel Cloudflare ---
--- Ejecutando Streamlit ---
Esperando generación de URL pública (aprox 10 seg)...

 TU APLICACIÓN ESTÁ LISTA. Haz clic aquí:
https://technology-desperate-exist-destination.trycloudflare.com

La aplicación seguirá corriendo. Detén la celda para finalizar.
